In [4]:
#http://127.0.0.1:8050/

import pandas as pd
import time
import dash
from dash import dcc, html, dash_table
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
import MetaTrader5 as mt5
import plotly.graph_objs as go

import sys
path_root = "D:/Documentos/Erik/TDR/TDR-Forex/"
sys.path.append(path_root)
import utils.mt5 as umt5

In [5]:
def create_selector(title, component, style=None):
    return dbc.Col(
        [
            html.H5(title, style={'textAlign': 'center', 'color': colors['text']}),
            html.Div(component, style=style or {'width': '100%', 'display': 'inline-block'})
        ]
    )

def get_empty_fig():
    empty_fig = go.Figure()
    empty_fig.update_layout(title_x=0.5,
                      paper_bgcolor='black',
                      plot_bgcolor='black',
                      xaxis_rangeslider_visible=False,
                      xaxis_showgrid=False, yaxis_showgrid=False,
                      font=dict(
                          color = 'black',
                          #family="Courier New, monospace",
                          size=10)
                     )
    return empty_fig

In [7]:
symbols_all = ["EURUSD","GBPUSD","USDJPY","USDCHF","AUDUSD","USDCAD","NZDUSD"]
timeframes = ["M1", "M5", "M15", "H1", "D1"]

tabs_styles = {
    'height': '44px'
}
tab_style = {
    'borderBottom': '1px solid #d6d6d6',
    'padding': '6px',
    'fontWeight': 'bold',
    'color': 'white',
    'backgroundColor': '#888888',
}

tab_selected_style = {
    'borderTop': '1px solid #d6d6d6',
    'borderBottom': '1px solid #d6d6d6',
    'backgroundColor': '#7f0000',
    'color': 'white',
    'padding': '6px'
}

colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}

tab1 = dcc.Tab(
    label='Gràfics en temps real', 
    children=[
        # Fila de selectors
        dbc.Row(
            [
                dbc.Col(
                    create_selector(
                        "Símbols",
                        dcc.Dropdown(
                            id='selector-symbol',
                            options=[{'label':name, 'value':name} for name in symbols_all],
                            value='EURUSD'
                        )
                    ),
                    width=3  # ancho relativo, de 1 a 12
                ),
                dbc.Col(
                    create_selector(
                        "Finestra",
                        dcc.Dropdown(
                            id='selector-period',
                            options=[{'label':name, 'value':name} for name in timeframes],
                            value='M1'
                        )
                    ),
                    width=2
                ),
            ],
            style={'padding': '0px 0px 10px 0px'}
        ),
        # Taula i gràfica
        dbc.Row([
            # Columna esquerra: Taula i hora
            dbc.Col([
                html.P(id="time"),
                dash_table.DataTable(
                    id='table-bid-ask',
                    style_as_list_view=True,
                    style_header={'backgroundColor': 'rgb(30, 30, 30)'},
                    style_cell={
                        'textAlign': 'center',
                        'backgroundColor': 'rgb(50, 50, 50)',
                        'color': 'white'
                    },
                    #sort_action='custom'
                ),
                dcc.Interval(
                    id='interval-component-table-bid-ask',
                    interval=5*1000,  # in milliseconds
                    n_intervals=0
                )
            ], width=5, style={"overflowX": "scroll", 'maxHeight': 500, 'padding-left': '30px'}),
        
            # Columna dreta: Gràfic
            dbc.Col([
                dcc.Graph(figure=get_empty_fig(), id='chart-currency'),
                dcc.Interval(
                    id='interval-component-chart-currency',
                    interval=60*1000,  # in milliseconds
                    n_intervals=0
                )
            ], width=7)
        ], style=tab_style)
    ], 
    style=tab_style, selected_style=tab_selected_style    
)


tab2 = dcc.Tab(
    label='Backtesting Results', 
    children=[
        ], 
    style=tab_style, selected_style=tab_selected_style
)

tab3 = dcc.Tab(
    label='Backtesting Trades', 
    children=[
        ], 
    style=tab_style, selected_style=tab_selected_style
)

tab4 = dcc.Tab(
    label='Live Trading', 
    children=[
        ], 
    style=tab_style, selected_style=tab_selected_style
)

body = dbc.Container(
    [
        dbc.Col([
            html.H1(children='Forex Trading Strategies',
                    style={'textAlign': 'center','color': colors['text']}),
            dcc.Tabs(id="tabs", children=[
                tab1,
                tab2,
                tab3,
                tab4,
                ], style={'padding': '20px 0px 10px 0px'})
        ],style={'width': '100%', "padding": "0%", 'display': 'inline-block'}),
    ], className="mt-4", style={"max-width": "98%"},
)

app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])
# Override HTML template rendered by Dash in order to change title and favicon
with open("templates/base.html") as base_html:
    app.index_string = base_html.read()

app.layout = html.Div(children=[body], style={'backgroundColor': colors['background']})

# Callback TAB1
@app.callback(
    [
        Output('table-bid-ask', 'data'),
        Output('table-bid-ask', 'columns'),
        Output('time', 'children')
    ],
    [Input('interval-component-table-bid-ask', 'n_intervals')]
)
def update_bid_ask(n):
    # Hora actual
    time_string = time.strftime("%Y-%m-%d %H:%M:%S")
    
    # DataFrame vacío
    table_bid_ask = pd.DataFrame(columns=['symbol', 'bid', 'ask', 'spread', 'volume', 'change'])
    
    # Inicializar MT5
    umt5.init_mt5()

    for i, symbol in enumerate(symbols_all):
        info = mt5.symbol_info(symbol)
        if info:
            digits = info.digits
            bid = round(info.bid, digits)
            ask = round(info.ask, digits)
            spread = round(info.spread / 10, 1)  # según tu código
            
            # Último dato diario
            rates = mt5.copy_rates_from_pos(symbol, mt5.TIMEFRAME_D1, 0, 1)
            if rates is not None and len(rates) > 0:
                r = rates[0]
                open_ = round(r['open'], digits)
                close_ = round(r['close'], digits)
                volume = r['tick_volume']
                daily_change = f"{round((close_/open_ - 1)*100,2)}%"
            else:
                open_ = close_ = volume = daily_change = None
            
            table_bid_ask.loc[i] = [symbol, bid, ask, spread, volume, daily_change]
    
    # Cerrar MT5
    mt5.shutdown()
    
    # Preparar datos para DataTable
    data_table = table_bid_ask.to_dict('records')
    columns_table = [{'id': c, 'name': c} for c in table_bid_ask.columns]
    
    return data_table, columns_table, time_string

@app.callback(
    Output('chart-currency', 'figure'),
    [
        Input('interval-component-chart-currency', 'n_intervals'),
        Input('selector-symbol', 'value'),
        Input('selector-period', 'value')
    ]
)
def update_line_chart(n, symbol, period):
    if not mt5.initialize():
        print("No se pudo inicializar MT5")
        return get_empty_fig()

    # Mapear periodos a MT5 TIMEFRAME
    timeframe_map = {
        'M1': mt5.TIMEFRAME_M1,
        'M5': mt5.TIMEFRAME_M5,
        'M15': mt5.TIMEFRAME_M15,
        'H1': mt5.TIMEFRAME_H1,
        'D1': mt5.TIMEFRAME_D1
    }
    tf = timeframe_map.get(period, mt5.TIMEFRAME_M1)

    # Últimas 200 barras
    rates = mt5.copy_rates_from_pos(symbol, tf, 0, 200)
    mt5.shutdown()

    if rates is None or len(rates) == 0:
        return get_empty_fig()

    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')

    # Calcular EMAs
    df['EMA18'] = df['close'].ewm(span=18, adjust=False).mean()
    df['EMA30'] = df['close'].ewm(span=30, adjust=False).mean()
    df['EMA200'] = df['close'].ewm(span=200, adjust=False).mean()

    # Crear figura de línea
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df['time'], y=df['close'], mode='lines', name='Close', line=dict(color='white')))
    fig.add_trace(go.Scatter(x=df['time'], y=df['EMA18'], mode='lines', name='EMA18', line=dict(color='yellow')))
    fig.add_trace(go.Scatter(x=df['time'], y=df['EMA30'], mode='lines', name='EMA30', line=dict(color='orange')))
    fig.add_trace(go.Scatter(x=df['time'], y=df['EMA200'], mode='lines', name='EMA200', line=dict(color='red')))

    # Layout sin grid
    fig.update_layout(
        title=f"{symbol} - {period}",
        xaxis_showgrid=False,
        yaxis_showgrid=False,
        paper_bgcolor='black',
        plot_bgcolor='black',
        font=dict(color='white')
    )

    return fig

app.run(debug=True, use_reloader=False, dev_tools_ui=True, dev_tools_props_check=True)